# Laboratorio 1: Detección de phishing

- Pedro Pablo Guzmán Mayen 22111
- Javier Andres Chen 22153

## Parte 1 - Ingeniería de características

### Exploración de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [1]:
import pandas as pd


df = pd.read_csv('dataset_pishing.csv')


df.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


Como vemos, el dataset contiene 2 columnas, status la cuál indica si el url es legítimo o no y la columna que contiene el url.

2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset?

In [2]:
df['status'].value_counts()

,count
status,
legitimate,5715
phishing,5715


El dataset si está balanceado pues hay la misma cantidad de urls maliciosas y no maliciosas.

**Derivacion de caracteristicas**

1. **¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?**

a) Detección en tiempo real sin descargar contenido:

* No requiere acceder al contenido de la página web, lo que evita riesgos de  seguridad
* Es significativamente más rápido ya que no necesita descargar HTML, CSS, JavaScript o imágenes

b) Eficiencia computacional:

* Requiere menos recursos del sistema
* Permite procesar grandes volúmenes de URLs rápidamente

c) Independencia de servicios de terceros:
* Evita retrasos de red causados por servicios externos
* Funciona incluso cuando servicios de terceros están caídos o son lentos


2. **¿Qué características de una URL son más prometedoras para la detección de phishing?**

a) Características de longitud:

* Longitud total de la URL
* Longitud del hostname/dominio

b) Entropía:

* Entropía de Shannon de caracteres no-alfanuméricos (propuesta clave)
Entropía relativa
* Mide la distribución/desorden de caracteres especiales

c) Características del dominio:

* Presencia de dirección IP en lugar de nombre de dominio
* Posición del TLD (si aparece en path o subdomain es sospechoso)
* Número de subdominios (phishing usa más)

d) Características estructurales:

* Uso de HTTPS (aunque atacantes ahora también lo usan - 78% según estudios)
* Presencia de puertos no estándar
* Uso de servicios de acortamiento de URLs

In [4]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
from collections import Counter
import math

In [5]:
# Cargar el dataset
df = pd.read_csv('dataset_pishing.csv')

# FUNCIONES DE EXTRACCIÓN DE CARACTERÍSTICAS

# 1. LONGITUD DE LA URL COMPLETA
def url_length(url):
    return len(url)

# 2. LONGITUD DEL DOMINIO/HOSTNAME
def domain_length(url):
    parsed = urlparse(url)
    return len(parsed.netloc)

# 3. LONGITUD DEL PATH
def path_length(url):
    parsed = urlparse(url)
    return len(parsed.path)

# 4. NÚMERO DE PUNTOS (.)
def count_dots(url):
    return url.count('.')

# 5. NÚMERO DE GUIONES (-)
def count_hyphens(url):
    return url.count('-')

# 6. NÚMERO DE CARACTERES ESPECIALES
def count_special_chars(url):
    special_chars = ['@', '?', '&', '=', '_', '~', '%', '*', '#', '$']
    return sum([url.count(char) for char in special_chars])

# 7. PRESENCIA DE DIRECCIÓN IP
def has_ip_address(url):
    parsed = urlparse(url)
    domain = parsed.netloc
    ip_pattern = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    return 1 if ip_pattern.search(domain) else 0

# 8. NÚMERO DE SUBDOMINIOS
def count_subdomains(url):
    parsed = urlparse(url)
    domain = parsed.netloc.split(':')[0]  # Remover puerto
    return max(0, domain.count('.') - 1)

# 9. USO DE HTTPS
def is_https(url):
    return 1 if url.startswith('https://') else 0

# 10. NÚMERO DE PARÁMETROS EN LA URL
def count_params(url):
    parsed = urlparse(url)
    query = parsed.query
    if not query:
        return 0
    return len(query.split('&'))

# 11. RATIO DE DÍGITOS EN LA URL
def digit_ratio(url):
    if len(url) == 0:
        return 0
    digits = sum(c.isdigit() for c in url)
    return digits / len(url)

# 12. PRESENCIA DE PALABRAS SENSIBLES
def has_sensitive_words(url):
    keywords = ['login', 'signin', 'account', 'update', 'secure', 'banking',
                'verify', 'confirm', 'password', 'paypal', 'apple']
    url_lower = url.lower()
    return sum(1 for keyword in keywords if keyword in url_lower)

# 13. PROFUNDIDAD DEL PATH
def path_depth(url):
    parsed = urlparse(url)
    path = parsed.path.strip('/')
    if not path:
        return 0
    return len(path.split('/'))

# 14. ENTROPÍA DE SHANNON
def shannon_entropy(url):
    # Extraer solo caracteres no-alfanuméricos
    non_alphanum = [c for c in url if not c.isalnum()]

    if len(non_alphanum) == 0:
        return 0

    # Calcular frecuencias
    freq = Counter(non_alphanum)
    total = len(non_alphanum)

    # Calcular entropía:
    entropy = 0
    for count in freq.values():
        p = count / total
        if p > 0:
            entropy -= p * math.log2(p)

    return entropy

# 15. ENTROPÍA RELATIVA
def relative_entropy(url):
    # Caracteres especiales comunes
    special_chars = ['.', '/', ':', '-', '?', '=', '&', '_', '~', '%']

    # Contar ocurrencias
    counts = {c: url.count(c) for c in special_chars}
    total = sum(counts.values())

    if total == 0:
        return 0

    # Distribución observada
    p = {c: counts[c] / total for c in special_chars}

    # Distribución uniforme
    q = 1 / len(special_chars)

    # Calcular KL divergence:
    kl_div = 0
    for char in special_chars:
        if p[char] > 0:
            kl_div += p[char] * math.log2(p[char] / q)

    return kl_div

# APLICAR TODAS LAS FUNCIONES AL DATASET

print("Extrayendo características de las URLs...")
print("Dataset original shape:", df.shape)

# Aplicar las 15 funciones
df['url_length'] = df['url'].apply(url_length)
df['domain_length'] = df['url'].apply(domain_length)
df['path_length'] = df['url'].apply(path_length)
df['count_dots'] = df['url'].apply(count_dots)
df['count_hyphens'] = df['url'].apply(count_hyphens)
df['count_special_chars'] = df['url'].apply(count_special_chars)
df['has_ip'] = df['url'].apply(has_ip_address)
df['num_subdomains'] = df['url'].apply(count_subdomains)
df['is_https'] = df['url'].apply(is_https)
df['num_params'] = df['url'].apply(count_params)
df['digit_ratio'] = df['url'].apply(digit_ratio)
df['sensitive_word_count'] = df['url'].apply(has_sensitive_words)
df['path_depth'] = df['url'].apply(path_depth)
df['shannon_entropy'] = df['url'].apply(shannon_entropy)
df['relative_entropy'] = df['url'].apply(relative_entropy)

print("\nDataset con características shape:", df.shape)
print("\nPrimeras 5 filas:")
print(df.head())

# Guardar dataset con características
df.to_csv('dataset_with_features.csv', index=False)
print("\n✓ Dataset guardado como 'dataset_with_features.csv'")

# Mostrar estadísticas
print("\n" + "="*80)
print("ESTADÍSTICAS DE LAS 15 CARACTERÍSTICAS")
print("="*80)
print(df.describe())

Extrayendo características de las URLs...
Dataset original shape: (11430, 2)

Dataset con características shape: (11430, 17)

Primeras 5 filas:
                                                 url      status  url_length  \
0              http://www.crestonwood.com/router.php  legitimate          37   
1  http://shadetreetechnology.com/V4/validation/a...    phishing          77   
2  https://support-appleld.com.secureupdate.duila...    phishing         126   
3                                 http://rgipt.ac.in  legitimate          18   
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate          55   

   domain_length  path_length  count_dots  count_hyphens  count_special_chars  \
0             19           11           3              0                    0   
1             23           47           1              0                    0   
2             50           20           4              1                    8   
3             11            0           2          